In [7]:
import pandas as pd
import numpy as np
import matplotlib
import seaborn as sns
import pystan
import fbprophet 

#built-in 
import datetime

print('pandas ==',pd.__version__)
print('numpy ==',np.__version__)
print('matplotlib ==',matplotlib.__version__)
print('seaborn ==',sns.__version__)
print('pystan ==',pystan.__version__)
print('fbprophet ==',fbprophet.__version__)

pandas == 1.2.3
numpy == 1.20.1
matplotlib == 3.3.4
seaborn == 0.11.1
pystan == 2.19.1.1
fbprophet == 0.7.1


# Funções

In [8]:
# =======================================================================================
## FUNÇÕES PARA A IMPORTAÇÃO DOS DADOS E TRANSFORMAR DATAS

def converte_variaveis_em_datas(dataframe):
    for variavel in dataframe.columns:
        if 'DATA' in variavel:
            try:
                dataframe[variavel] = pd.to_datetime(dataframe[variavel], format='%d/%m/%Y')
            
            except:
                print(f'Variável "{variavel}" contém um erro e conversão/formatação')
                pass
        else:
            pass 
    return dataframe


def tira_variaveis_IBGE(dataframe):
    dataframe = dataframe.drop(['IBGE_RES_PR','IBGE_ATEND_PR'], axis='columns')
    return dataframe


def cria_variavel_caso_confirmado_no_dia(dataframe):
    dataframe['CASO_CONFIRMADO_NO_DIA'] = 1
    return dataframe


def replace_nas_variaveis_obito_status(dataframe):

    dataframe['OBITO'] = dataframe['OBITO'].replace('SIM','Sim')
    dataframe['OBITO'] = dataframe['OBITO'].replace('Não','Nao')

    dataframe['OBITO'] = dataframe['OBITO'].replace('Sim',1)
    dataframe['OBITO'] = dataframe['OBITO'].replace('Nao',0)

    dataframe['STATUS'] = dataframe['STATUS'].replace('Recuperado','recuperado')
    dataframe['STATUS'] = dataframe['STATUS'].replace('recuperado', 1)
    dataframe['STATUS'] = dataframe['STATUS'].replace('nan', 0)

    return dataframe
    

def pre_processamento(dataframe):
    dataframe = converte_variaveis_em_datas(dataframe)
    dataframe = tira_variaveis_IBGE(dataframe)
    dataframe = cria_variavel_caso_confirmado_no_dia(dataframe)
    dataframe = replace_nas_variaveis_obito_status(dataframe)
    dataframe = dataframe.set_index('DATA_CONFIRMACAO_DIVULGACAO')
    dataframe = dataframe.sort_index()
    return dataframe

def baixa_base_de_dados_casos_gerais(ano,mes,dia):
    """
    Esta função baixa a base de dados disponível no ano, mes e dia.
    retorna um dataframe do pandas com os dados disponpiveis.
    """
    data = datetime.date(ano, mes, dia)
    ano = str(data.year)

    if mes != 10 and mes != 11 and mes != 12:
        mes = '0'+str(data.month)
    else:
        mes = str(data.month)

    if dia in [1,2,3,4,6,7,8,9]:
        dia = '0'+str(data.day)

    ano_mes = ano+'-'+mes

    if ano == '2020':
        arquivo = f'INFORME_EPIDEMIOLOGICO_{dia}_{mes}_GERAL.csv'
    elif ano == '2021':
        arquivo = f'informe_epidemiologico_{dia}_{mes}_{ano}_geral.csv'
    # Podem acontecer as seguintes variações no site do estado:
    # informe_epidemiologico_{dia}_{mes}_{ano}_geral.csv
    # informe_epidemiologico_{dia}_{mes}_geral.csv
    # INFORME_EPIDEMIOLOGICO_{dia}_{mes}_GERAL.csv



    dominio = 'https://www.saude.pr.gov.br'
    caminho = f'/sites/default/arquivos_restritos/files/documento/{ano_mes}/'
    try:

        url = dominio+caminho+arquivo
        base_de_dados = pd.read_csv(url, sep=';')
        base_de_dados = pre_processamento(base_de_dados)
        
    
    except:
        raise Exception('Não Tem dados disponíveis neste dia.')

    return base_de_dados

# =======================================================================================
## FUNÇÕES PARA SELEÇÃO 




# Importando os Dados 

In [24]:
hoje = datetime.date.today()
informe_covid = baixa_base_de_dados_casos_gerais(hoje.year, hoje.month, hoje.day - 4)

Variável "DATA_INICIO_SINTOMAS" contém um erro e conversão/formatação


In [25]:
informe_covid

,SEXO,IDADE_ORIGINAL,MUN_RESIDENCIA,MUN_ATENDIMENTO,LABORATORIO,DATA_DIAGNOSTICO,DATA_INICIO_SINTOMAS,OBITO,DATA_OBITO,DATA_OBITO_DIVULGACAO,STATUS,DATA_RECUPERADO_DIVULGACAO,FONTE_DADO_RECUPERADO,CASO_CONFIRMADO_NO_DIA
DATA_CONFIRMACAO_DIVULGACAO,,,,,,,,,,,,,,
2020-03-12,F,26,CURITIBA,CURITIBA,Genoprimer,2020-03-18,15/03/2020,0.0,NaT,NaT,1.0,NaT,NaN,1
2020-03-12,M,43,CURITIBA,CURITIBA,LACEN,2020-03-12,07/03/2020,0.0,NaT,NaT,1.0,NaT,NaN,1
2020-03-12,M,28,SAO PAULO/SP,CURITIBA,Genoprimer,2020-03-16,NaN,0.0,NaT,NaT,1.0,2020-08-05,SMS,1
2020-03-12,F,47,CIANORTE,CIANORTE,LACEN,2020-03-12,NaN,0.0,NaT,NaT,1.0,NaT,Notificações,1
2020-03-12,M,15,CURITIBA,CURITIBA,LACEN,2020-03-12,08/03/2020,0.0,NaT,NaT,1.0,NaT,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-04,F,85,FOZ DO IGUACU,FOZ DO IGUACU,Teste Rápido Antígeno (Imunocromatografia),2021-03-02,25/02/2021,NaN,NaT,NaT,NaN,NaT,NaN,1
2021-03-04,M,31,MARINGA,MARINGA,"COVID-19, Biologia Molecular",2021-03-02,21/02/2021,NaN,NaT,NaT,NaN,NaT,NaN,1
2021-03-04,M,30,GUAIRA,GUAIRA,"COVID-19, Biologia Molecular",2021-02-08,31/01/2021,0.0,NaT,NaT,1.0,2021-03-04,NOTIFICA COVID,1


In [26]:
dados_pg = informe_covid.query('MUN_ATENDIMENTO == "PONTA GROSSA"')

In [27]:
dados_pg.sort_index()

,SEXO,IDADE_ORIGINAL,MUN_RESIDENCIA,MUN_ATENDIMENTO,LABORATORIO,DATA_DIAGNOSTICO,DATA_INICIO_SINTOMAS,OBITO,DATA_OBITO,DATA_OBITO_DIVULGACAO,STATUS,DATA_RECUPERADO_DIVULGACAO,FONTE_DADO_RECUPERADO,CASO_CONFIRMADO_NO_DIA
DATA_CONFIRMACAO_DIVULGACAO,,,,,,,,,,,,,,
2020-03-21,M,33,PONTA GROSSA,PONTA GROSSA,LACEN,2020-03-21,17/03/2020,0.0,NaT,NaT,1.0,NaT,Epidemio Municipal,1
2020-03-25,F,61,PONTA GROSSA,PONTA GROSSA,LACEN,2020-03-25,12/03/2020,0.0,NaT,NaT,1.0,NaT,Epidemio Municipal,1
2020-03-25,F,30,PONTA GROSSA,PONTA GROSSA,LACEN,2020-03-25,15/03/2020,0.0,NaT,NaT,1.0,NaT,Epidemio Municipal,1
2020-04-04,M,54,PONTA GROSSA,PONTA GROSSA,DB,2020-03-31,NaN,0.0,NaT,NaT,1.0,NaT,SMS Ponta Grossa,1
2020-04-04,F,71,CASTRO,PONTA GROSSA,DB,2020-03-29,NaN,0.0,NaT,NaT,1.0,NaT,SMS Castro,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-04,M,25,PONTA GROSSA,PONTA GROSSA,"COVID-19, Biologia Molecular",2021-03-01,23/02/2021,NaN,NaT,NaT,NaN,NaT,NaN,1
2021-03-04,M,37,GUARAPUAVA,PONTA GROSSA,"COVID-19, Biologia Molecular",2021-01-13,NaN,NaN,NaT,NaT,NaN,NaT,NaN,1
2021-03-04,M,66,PONTA GROSSA,PONTA GROSSA,Teste Rápido Antígeno (Imunocromatografia),2021-03-03,NaN,NaN,NaT,NaT,NaN,NaT,NaN,1


In [59]:
def seleciona_cidade(cidade):
    """
    Esta função retona todos os registros
    cujo o atendimento ocorreu na cidade 
    passada como parâmetro.
    Retorna um dataframe.
    """
    dados_cidade = informe_covid.query(f'MUN_ATENDIMENTO == "{cidade.upper()}"')
    return dados_cidade



In [55]:
dado_cidade = seleciona_cidade('ponta grossa')
dado_cidade

,SEXO,IDADE_ORIGINAL,MUN_RESIDENCIA,MUN_ATENDIMENTO,LABORATORIO,DATA_DIAGNOSTICO,DATA_INICIO_SINTOMAS,OBITO,DATA_OBITO,DATA_OBITO_DIVULGACAO,STATUS,DATA_RECUPERADO_DIVULGACAO,FONTE_DADO_RECUPERADO,CASO_CONFIRMADO_NO_DIA
DATA_CONFIRMACAO_DIVULGACAO,,,,,,,,,,,,,,
2020-03-21,M,33,PONTA GROSSA,PONTA GROSSA,LACEN,2020-03-21,17/03/2020,0.0,NaT,NaT,1.0,NaT,Epidemio Municipal,1
2020-03-25,F,61,PONTA GROSSA,PONTA GROSSA,LACEN,2020-03-25,12/03/2020,0.0,NaT,NaT,1.0,NaT,Epidemio Municipal,1
2020-03-25,F,30,PONTA GROSSA,PONTA GROSSA,LACEN,2020-03-25,15/03/2020,0.0,NaT,NaT,1.0,NaT,Epidemio Municipal,1
2020-04-04,M,54,PONTA GROSSA,PONTA GROSSA,DB,2020-03-31,NaN,0.0,NaT,NaT,1.0,NaT,SMS Ponta Grossa,1
2020-04-04,F,71,CASTRO,PONTA GROSSA,DB,2020-03-29,NaN,0.0,NaT,NaT,1.0,NaT,SMS Castro,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-04,M,25,PONTA GROSSA,PONTA GROSSA,"COVID-19, Biologia Molecular",2021-03-01,23/02/2021,NaN,NaT,NaT,NaN,NaT,NaN,1
2021-03-04,M,37,GUARAPUAVA,PONTA GROSSA,"COVID-19, Biologia Molecular",2021-01-13,NaN,NaN,NaT,NaT,NaN,NaT,NaN,1
2021-03-04,M,66,PONTA GROSSA,PONTA GROSSA,Teste Rápido Antígeno (Imunocromatografia),2021-03-03,NaN,NaN,NaT,NaT,NaN,NaT,NaN,1
